### L_Game_Team

In [36]:
import pandas as pd
from datetime import datetime
import sqlite3
import hashlib

In [37]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name_1='Source_L_Game_Team'
table_name_2='L_Game_Team'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
drop_query_1 = f"DROP TABLE IF EXISTS {table_name_1};"
drop_query_2 = f"DROP TABLE IF EXISTS {table_name_2};"
cursor.execute(drop_query_1)
cursor.execute(drop_query_2)
connection.commit()
connection.close()

Get Game Team delais from "D:/AFL_GRP/DataBase/AFLStats.db"."ScrapedPlayerStatistics"

In [38]:
data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
connection=sqlite3.connect(data_base)
query=f''' 
select
       year,round,team
       from ScrapedPlayerStatistics 
       where 1=1
       group by year,round,team
      '''
df=pd.read_sql(query,connection)
connection.close()


In [39]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='Source_L_Game_Team'
connection=sqlite3.connect(database=data_base)
df.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()

In [40]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(data_base)
query=f''' 
select
     ht.Team_hashkey
      ,hg.Game_Hashkey
      ,slgt.*
from  Source_L_Game_Team slgt
join  H_Team ht
on    slgt.Team=ht.team
join  H_Game hg
on    slgt.year=hg.year
and   slgt.round=hg.round
and   slgt.team=hg.team
where 1=1
union all
select
      ht.Team_hashkey
      ,hg.Game_Hashkey
      ,slgt.*
from  Source_L_Game_Team slgt
join  H_Team ht
on    slgt.Team=ht.team
join  H_Game hg
on    slgt.year=hg.year
and   slgt.round=hg.round
and   slgt.team=hg.opponent
where 1=1
'''
df=pd.read_sql(query,connection)
connection.close()


In [41]:
def hash_row(row):
    combined = str(row['Year']) + str(row['Round']) + str(row['Team']) 
    # Generate a hash using SHA-256
    return hashlib.sha256(combined.encode()).hexdigest()

df['GameTeam_Hashkey']=df['GameTeam_Hashkey'] = df.apply(hash_row, axis=1)
df['Load_Date']=datetime.now().date()
df['Record_Source']='ScrapedPlayerStatistics'

df=df[['GameTeam_Hashkey','Game_Hashkey','Team_Hashkey','Load_Date','Record_Source']]


In [42]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='L_Game_Team'
connection=sqlite3.connect(database=data_base)
df.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()

In [43]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query=f'''select * from l_game_team '''
df=pd.read_sql(query,connection)
connection.close()
df

,GameTeam_Hashkey,Game_Hashkey,Team_Hashkey,Load_Date,Record_Source
0,90dedc2d235615b21abd8087d24315a15d4cba475790a0...,ce7aecda92f3399d6ba840e87bcbb3cad13659ced17718...,9bf6f67741ec188893958d39bf09e6532600ec73778d6f...,2024-10-13,ScrapedPlayerStatistics
1,154f2d1a57c1596f48881f0a7f58cd0bfe42e57a21d35c...,067697c47c8a142034798ebbe1b6ad2f7f37b8bb998b59...,e87601989d7d2997b3e9ebc455fb8fc4f6f65f46151e56...,2024-10-13,ScrapedPlayerStatistics
2,d0a46f8f771c6908e5910e054256a5169a0e6ab8ac4dae...,1fc4b78a92bd2d6a02584217edd08700e54108067d50b0...,ce344a418127968f2bd07ada86c8cf3808eea524c399a0...,2024-10-13,ScrapedPlayerStatistics
3,8f9f7675f725f0b5bd76210bbff5403dfbcc90666f9264...,103a105d6c94964814d311d0983d964180f09666ef2dc0...,ce344a418127968f2bd07ada86c8cf3808eea524c399a0...,2024-10-13,ScrapedPlayerStatistics
4,6cce5f17b7d2cd285ed5c39790144e22e0c1dc19477b9c...,e05d87b7bf069ec8938b84c6dcb6baa3aacba93fedbc02...,7021db8687be1dfdbf5705ff61ae83a79d2c04b3d668a8...,2024-10-13,ScrapedPlayerStatistics
...,...,...,...,...,...
9783,5c7e976284de840505c2dd5a3e6806cffd9edf1411823b...,45dcabb15a9103ab58a73a8a25b29c249a4a64808d71ae...,86a7f47b333770c6e85f03ee3caa6ef9dfebfe2cb3444b...,2024-10-13,ScrapedPlayerStatistics
9784,c4887413f24962b74b6ff3f708bdf079e5d016e4f86fa3...,107ac7ba4037d6ccd5671f3b187028ce69354e77155984...,1ae55232c15b65f071015b34ba493dc4c0057cef485968...,2024-10-13,ScrapedPlayerStatistics
9785,82447edb263228f0a89966f5315dba9353dbc2cdd72b2f...,2d88a475049a1ec7c5816bb90b5df9f18ef0b3dfb044d2...,fc233d527d220e5bea38fcff1ce0dcad5c371c81e0a800...,2024-10-13,ScrapedPlayerStatistics
9786,1d88d5c2dbe3bf9b1168af12a66f6d67404c887168448b...,24f70b6581fd27418323e1e9a6d176ed1df6f4e3cf9b65...,9bf6f67741ec188893958d39bf09e6532600ec73778d6f...,2024-10-13,ScrapedPlayerStatistics
